In [2]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True


fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.3: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers via:
`pip uninstall transformers -y && pip install --upgrade --no-cache-dir "git+https://github.com/huggingface/transformers.git"`


In [4]:
!pip install datasets

In [5]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""



In [6]:
from datasets import load_dataset
import re

In [7]:
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    texts = []
    for text, summary in zip(examples["text"], examples["summary"]):
        parts = re.split(r'[:.\n]', text, maxsplit=1)
        if len(parts) == 2:
            command, input_text = parts[0].strip(), parts[1].strip()
            formatted_text = alpaca_prompt.format(command, input_text, summary) + EOS_TOKEN
            texts.append(formatted_text)
        else:
            continue
    return {"text": texts}

dataset = load_dataset("RussianNLP/Mixed-Summarization-Dataset")
dataset['train'] = dataset['train'].map(formatting_prompts_func, batched=True)
dataset['test'] = dataset['test'].map(formatting_prompts_func, batched=True)

README.md:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/538M [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/605k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/197561 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/258 [00:00<?, ? examples/s]

Map:   0%|          | 0/197561 [00:00<?, ? examples/s]

Map:   0%|          | 0/258 [00:00<?, ? examples/s]

In [8]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # any number > 0 (8, 16, 32, 64, 128)
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.10.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to="wandb"
    ),
)

Map (num_proc=2):   0%|          | 0/197561 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [10]:
trainer_stats = trainer.train()

**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers and Unsloth!


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 197,561 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,1.692000
2,1.831200
3,1.982500
4,1.836300
5,1.762800
6,1.732200
7,1.554700
8,1.717700
9,1.782500
10,1.566000


In [11]:
with open('/content/full_podcast.txt', 'r', encoding='utf-8') as file:
    content = file.readlines()

for line in content[:5]:
    print(line.strip())


Ведущий 1 (Анна):
Здравствуйте, дорогие слушатели! Добро пожаловать в наш подкаст "Инвестиции в Москве". Я – Анна, ваш ведущий, и сегодня у нас очень интересная тема – инвестиции в недвижимость, а особенно в парковочные места. Со мной в студии мой коллега и эксперт в области недвижимости, Михаил. Привет, Михаил!

Ведущий 2 (Михаил):
Привет, Анна! Рад быть здесь и обсудить столь актуальный вопрос. Инвестиции в недвижимость всегда были популярны, но сегодня мы особенно сосредоточимся на парковочных местах в Москве. Это действительно перспективное направление, и я уверен, наши слушатели найдут его полезным.


In [42]:
# alpaca_prompt
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Расскажи основной смысл до 300 символов", # instruction
        content, # input
        "", # output
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
response = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 100)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Расскажи основной смысл до 300 символов

### Input:
['Ведущий 1 (Анна):\n', 'Здравствуйте, дорогие слушатели! Добро пожаловать в наш подкаст "Инвестиции в Москве". Я – Анна, ваш ведущий, и сегодня у нас очень интересная тема – инвестиции в недвижимость, а особенно в парковочные места. Со мной в студии мой коллега и эксперт в области недвижимости, Михаил. Привет, Михаил!\n', '\n', 'Ведущий 2 (Михаил):\n', 'Привет, Анна! Рад быть здесь и обсудить столь актуальный вопрос. Инвестиции в недвижимость всегда были популярны, но сегодня мы особенно сосредоточимся на парковочных местах в Москве. Это действительно перспективное направление, и я уверен, наши слушатели найдут его полезным.\n', '\n', 'Анна:\n', 'Абсолютно согласна. Начнем с того, почему недвижимость в целом считается надежным видом инвестиций,

In [43]:
length = len("Инвестиции в парковочные места в Москве – это перспективный рынок с высокой доходностью и относительно низкими рисками. Недвижимость в целом считается надежным видом инвестиций, а парковочные места в Москве являются особенно привлекательным объектом для инвесторов.<|end_of_text|>")
print(length)

280


In [ ]:
generated_text = tokenizer.decode(response[0], skip_special_tokens=True)
print(generated_text)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
О чем беседуют люди 

### Input:
['Ведущий 1 (Анна):\n', 'Здравствуйте, дорогие слушатели! Добро пожаловать в наш подкаст "Инвестиции в Москве". Я – Анна, ваш ведущий, и сегодня у нас очень интересная тема – инвестиции в недвижимость, а особенно в парковочные места. Со мной в студии мой коллега и эксперт в области недвижимости, Михаил. Привет, Михаил!\n', '\n', 'Ведущий 2 (Михаил):\n', 'Привет, Анна! Рад быть здесь и обсудить столь актуальный вопрос. Инвестиции в недвижимость всегда были популярны, но сегодня мы особенно сосредоточимся на парковочных местах в Москве. Это действительно перспективное направление, и я уверен, наши слушатели найдут его полезным.\n', '\n', 'Анна:\n', 'Абсолютно согласна. Начнем с того, почему недвижимость в целом считается надежным видом инвестиций, а потом углубимся в тему парковочны

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
О чем диалог

### Input:
['Ведущий 1 (Анна):\n', 'Здравствуйте, дорогие слушатели! Добро пожаловать в наш подкаст "Инвестиции в Москве". Я – Анна, ваш ведущий, и сегодня у нас очень интересная тема – инвестиции в недвижимость, а особенно в парковочные места. Со мной в студии мой коллега и эксперт в области недвижимости, Михаил. Привет, Михаил!\n', '\n', 'Ведущий 2 (Михаил):\n', 'Привет, Анна! Рад быть здесь и обсудить столь актуальный вопрос. Инвестиции в недвижимость всегда были популярны, но сегодня мы особенно сосредоточимся на парковочных местах в Москве. Это действительно перспективное направление, и я уверен, наши слушатели найдут его полезным.\n', '\n', 'Анна:\n', 'Абсолютно согласна. Начнем с того, почему недвижимость в целом считается надежным видом инвестиций, а потом углубимся в тему парковочных мест. Михаил, расскажи, пожалуйста, о текущей ситуации на рынке недвижимости Москвы.\n', '\n', 'Михаил:\n', 'Конечно. Москва – это один из крупнейших и динамично развивающихся городов России. Спрос на недвижимость здесь стабильно высокий из-за постоянного притока людей, бизнесов и инвесторов. Цены на жилье и коммерческую недвижимость постепенно растут, что делает их привлекательными для инвесторов. Кроме того, Москва продолжает расширяться, появляются новые районы с развитой инфраструктурой, что также стимулирует рост цен на недвижимость.\n', '\n', 'Анна:\n', 'Это действительно впечатляет. А как насчет парковочных мест? Почему именно они стали таким привлекательным объектом для инвестиций?\n', '\n', 'Михаил:\n', 'Отличный вопрос. Парковочные места в Москве становятся всё более востребованными по нескольким причинам. Во-первых, количество автомобилей в городе постоянно растет, и спрос на парковочные места превышает предложение. Во-вторых, многие жилые комплексы и бизнес-центры сталкиваются с проблемой нехватки парковочных мест для своих жильцов и сотрудников. Это создает отличный рынок для инвесторов, поскольку парковочные места можно сдавать в аренду или перепродавать с хорошей наценкой.\n', '\n', 'Анна:\n', 'Понятно. Какие преимущества дает инвестиция в парковочные места по сравнению с другими видами недвижимости?\n', '\n', 'Михаил:\n', 'Во-первых, парковочные места требуют меньших первоначальных вложений по сравнению с жилой или коммерческой недвижимостью. Это снижает порог входа для инвесторов. Во-вторых, они менее подвержены колебаниям рынка. Даже в условиях экономической нестабильности автомобили остаются востребованными, и люди продолжают нуждаться в парковочных местах. Третье преимущество – высокая доходность. Аренда парковочного места может приносить стабильный доход с минимальными затратами на содержание.\n', '\n', 'Анна:\n', 'Это звучит очень привлекательно. А как насчет рисков? Какие потенциальные риски существуют при инвестировании в парковочные места?\n', '\n', 'Михаил:\n', 'Конечно, как и в любом инвестировании, здесь есть свои риски. Один из них – изменение законодательства. Если, например, будут введены новые нормы или ограничения на использование парковочных мест, это может повлиять на доходность. Второй риск – конкуренция. В некоторых районах Москвы предложение парковочных мест уже достаточно высоко, что может снизить арендные ставки. Третий риск – технические проблемы, такие как необходимость ремонта или модернизации парковочного места, что может потребовать дополнительных затрат.\n', '\n', 'Анна:\n', 'Понятно. Какие меры можно предпринять, чтобы минимизировать эти риски?\n', '\n', 'Михаил:\n', 'Во-первых, важно тщательно выбирать локацию. Парковочные места в центральных районах Москвы или вблизи крупных бизнес-центров и торговых комплексов обычно более востребованы и менее подвержены колебаниям рынка. Во-вторых, стоит подписывать долгосрочные договоры аренды с надежными арендаторами, что обеспечит стабильный доход. Также полезно следить за изменениями в законодательстве и адаптироваться к ним заранее. И, конечно, регулярное техническое обслуживание парковочных мест поможет избежать неожиданных расходов.\n', '\n', 'Анна:\n', 'Отличные советы. А какова текущая доходность от инвестиций в парковочные места в Москве?\n', '\n', 'Михаил:\n', 'Доходность может варьироваться в зависимости от местоположения и условий аренды, но в среднем она составляет от 8% до 12% годовых. В некоторых премиальных районах доходность может быть еще выше. Кроме того, стоимость парковочного места со временем, как правило, растет, что позволяет не только получать доход от аренды, но и увеличивать капитализацию инвестиций.\n', '\n', 'Анна:\n', 'Это действительно впечатляющие цифры. А как вы оцениваете перспективы рынка парковочных мест в ближайшие 5-10 лет?\n', '\n', 'Михаил:\n', 'Я считаю, что рынок парковочных мест в Москве будет продолжать расти. Несмотря на развитие общественного транспорта и экологические инициативы, автомобили останутся важной частью городской инфраструктуры. Кроме того, с ростом числа жителей и автомобилей в городе увеличится и потребность в парковочных местах. Также можно ожидать появления новых технологий, таких как автоматизированные парковки и системы управления, что повысит эффективность использования парковочных пространств и, соответственно, их доходность.\n', '\n', 'Анна:\n', 'Интересно. А как насчет экологических аспектов? Ведь увеличение числа автомобилей негативно сказывается на экологии. Есть ли здесь какие-то решения, которые могут сочетать инвестиции в парковочные места с заботой об окружающей среде?\n', '\n', 'Михаил:\n', 'Да, безусловно. Например, можно инвестировать в строительство парковок с зарядными станциями для электромобилей, что поддерживает экологические инициативы и привлекает владельцев таких автомобилей. Также можно использовать экологически чистые материалы при строительстве и оборудовании парковок, а также внедрять системы энергосбережения, такие как солнечные панели для освещения. Такие меры не только способствуют защите окружающей среды, но и делают парковочные места более привлекательными для арендаторов, что может повысить их доходность.\n', '\n', 'Анна:\n', 'Это отличный пример того, как можно сочетать прибыль с ответственным отношением к природе. А что вы думаете о цифровизации и автоматизации в сфере парковочных мест? Как это влияет на инвестиции?\n', '\n', 'Михаил:\n', 'Цифровизация и автоматизация играют важную роль в повышении эффективности управления парковочными местами. Системы автоматического управления, онлайн-бронирования и оплаты, а также интеграция с мобильными приложениями делают использование парковок более удобным для пользователей. Для инвесторов это означает возможность оптимизировать процесс сдачи парковочных мест в аренду, снизить затраты на управление и увеличить доходность. Кроме того, такие технологии могут привлекать больше клиентов и повышать конкурентоспособность ваших парковочных мест на рынке.\n', '\n', 'Анна:\n', 'Понятно. А каковы основные критерии выбора парковочного места для инвестиций? На что стоит обращать внимание?\n', '\n', 'Михаил:\n', 'Первое – местоположение. Как я уже упоминал, центральные районы, деловые центры, рядом с торговыми площадями и транспортными узлами всегда пользуются высоким спросом. Второе – инфраструктура. Наличие удобных подъездных путей, безопасность, освещение и чистота – важные факторы для арендаторов. Третье – юридическая чистота объекта. Необходимо убедиться, что парковочное место зарегистрировано правильно и нет юридических препятствий для его сдачи в аренду. Четвертое – возможность расширения или модернизации. Важно, чтобы парковка могла адаптироваться к изменяющимся потребностям рынка.\n', '\n', 'Анна:\n', 'Отлично. А какие юридические аспекты нужно учитывать при инвестировании в парковочные места в Москве?\n', '\n', 'Михаил:\n', 'Прежде всего, нужно удостовериться в правовом статусе парковочного места. Если это часть жилого комплекса, необходимо проверить, разрешено ли сдавать парковку в коммерческую аренду. Также важно оформить договор купли-продажи правильно, чтобы избежать споров в будущем. Рекомендуется привлекать юристов, специализирующихся на недвижимости, для проверки всех документов. Еще один момент – налогообложение. Доход от аренды парковочного места подлежит налогообложению, поэтому стоит заранее продумать налоговую стратегию и, возможно, воспользоваться услугами бухгалтера.\n', '\n', 'Анна:\n', 'Спасибо за подробный ответ. Теперь давайте поговорим о сравнении инвестиций в парковочные места с другими видами недвижимости, например, с жилыми или коммерческими объектами. В чем основные различия?\n', '\n', 'Михаил:\n', 'Основные различия заключаются в размере вложений, сроках окупаемости и рисках. Жилая недвижимость обычно требует больших первоначальных вложений и имеет более длительные сроки окупаемости, но при этом предоставляет стабильный доход от аренды. Коммерческая недвижимость, такая как офисы или торговые площади, может предложить более высокую доходность, но также сопряжена с большими рисками, например, с арендными дефолтами. Парковочные места, как мы уже обсуждали, требуют меньших вложений, имеют более короткие сроки окупаемости и менее подвержены экономическим колебаниям. Это делает их привлекательным вариантом для инвесторов, особенно для тех, кто только начинает свой путь в недвижимости.\n', '\n', 'Анна:\n', 'Это очень полезная информация. А какие стратегии инвестирования в парковочные места вы бы рекомендовали начинающим инвесторам?\n', '\n', 'Михаил:\n', 'Для начинающих инвесторов я бы рекомендовал начать с покупки парковочных мест в хорошо зарекомендовавших себя районах с высоким спросом. Важно проводить тщательный анализ рынка и выбирать объекты с минимальными рисками. Также стоит рассмотреть возможность покупки нескольких парковочных мест для диверсификации портфеля. Еще одна стратегия – сотрудничество с управляющими компаниями, которые помогут с арендой и техническим обслуживанием парковочных мест. Кроме того, инвесторы могут рассмотреть варианты долгосрочной аренды, что обеспечит стабильный доход и снизит риски, связанные с частой сменой арендаторов.\n', '\n', 'Анна:\n', 'Отличные советы. А как насчет управления парковочными местами? Нужно ли инвестору лично заниматься этим или можно передать эту задачу профессионалам?\n', '\n', 'Михаил:\n', 'Это зависит от масштабов инвестиций и личных предпочтений. Если инвестор покупает несколько парковочных мест, управление ими может стать утомительным, поэтому передача этой задачи профессиональным управляющим компаниям будет разумным решением. Такие компании могут обеспечить эффективное управление, регулярное обслуживание и своевременную аренду парковочных мест. Для небольших инвестиций можно рассмотреть вариант самостоятельного управления, но это требует времени и усилий. В любом случае, важно выбрать надежного управляющего партнера, который будет работать в интересах инвестора и обеспечивать максимальную доходность.\n', '\n', 'Анна:\n', 'Понятно. А как технологии, такие как IoT и умные системы, влияют на управление парковочными местами?\n', '\n', 'Михаил:\n', 'Технологии, такие как Интернет вещей (IoT) и умные системы управления, значительно улучшают процессы управления парковочными местами. Например, с помощью сенсоров можно отслеживать наличие свободных мест в реальном времени и информировать арендаторов через мобильные приложения. Это повышает удобство использования парковки и увеличивает её привлекательность. Умные системы управления позволяют автоматизировать процессы оплаты, бронирования и мониторинга, что снижает затраты на управление и минимизирует ошибки. Кроме того, такие технологии могут способствовать повышению безопасности, например, за счет видеонаблюдения и контроля доступа. Внедрение этих технологий делает парковочные места более конкурентоспособными и способствует увеличению доходности инвестиций.\n', '\n', 'Анна:\n', 'Это действительно впечатляет. А что вы думаете о перспективах электромобилей и их влиянии на рынок парковочных мест?\n', '\n', 'Михаил:\n', 'Рост популярности электромобилей открывает новые возможности для инвесторов в парковочные места. Установка зарядных станций становится важным преимуществом, поскольку владельцы электромобилей будут искать парковки с такими удобствами. Это может повысить стоимость парковочных мест и привлечь дополнительных арендаторов. Более того, такие парковочные места могут получать дополнительные доходы от оплаты за зарядку, что увеличивает общую доходность инвестиций. В долгосрочной перспективе переход на электромобили будет способствовать развитию инфраструктуры для их обслуживания, и парковочные места, оборудованные зарядными станциями, будут пользоваться постоянным спросом.\n', '\n', 'Анна:\n', 'Замечательно. А как насчет сезонных колебаний спроса? Есть ли у парковочных мест такая проблема?\n', '\n', 'Михаил:\n', 'В целом, спрос на парковочные места в Москве остается относительно стабильным круглый год. Однако могут быть небольшие сезонные колебания, связанные с изменением количества туристов или погодными условиями. Например, зимой может увеличиться спрос на крытые парковки из-за неблагоприятных погодных условий. Тем не менее, такие колебания незначительны по сравнению с общим стабильным спросом на парковочные места. Для минимизации влияния сезонных изменений важно предлагать различные виды парковок и гибко реагировать на потребности арендаторов.\n', '\n', 'Анна:\n', 'Это утешает. А какова средняя цена парковочного места в Москве и как она варьируется по разным районам?\n', '\n', 'Михаил:\n', 'Средняя цена парковочного места в Москве может сильно варьироваться в зависимости от района и типа парковки. В центральных районах стоимость может начинаться от 10-15 миллионов рублей за место, тогда как в отдаленных районах цены могут быть ниже – от 5 до 8 миллионов рублей. Также важно учитывать тип парковки: крытые парковки обычно стоят дороже открытых. Вблизи крупных бизнес-центров и торговых комплексов цены будут выше из-за большего спроса. Стоимость аренды парковочного места также варьируется, но в среднем она составляет от 5 до 10 тысяч рублей в месяц в зависимости от местоположения и условий.\n', '\n', 'Анна:\n', 'Спасибо за подробную информацию. А как инвесторам лучше финансировать покупку парковочных мест? Есть ли какие-то специальные программы или кредиты для этого?\n', '\n', 'Михаил:\n', 'Для финансирования покупки парковочных мест инвесторы могут воспользоваться различными вариантами. Один из них – ипотечные кредиты на коммерческую недвижимость, включая парковочные места. Банки предлагают специальные условия для таких инвестиций, но процентные ставки могут быть выше, чем по ипотеке на жилую недвижимость. Другой вариант – привлечение инвесторов или партнеров для совместного приобретения объектов. Также можно рассмотреть использование собственных средств или реинвестирование прибыли от других инвестиций. Важно тщательно анализировать условия кредитования и выбирать наиболее выгодные предложения, чтобы обеспечить максимальную доходность и минимальные риски.\n', '\n', 'Анна:\n', 'Понятно. А есть ли у вас примеры успешных инвестиций в парковочные места в Москве, которые вы могли бы поделиться?\n', '\n', 'Михаил:\n', 'Да, конечно. Один из примеров – инвестиция в парковочные места в районе Москва-Сити. Этот район постоянно развивается, количество офисных и жилых зданий растет, что увеличивает спрос на парковочные места. Инвестор, купивший несколько парковочных мест в этом районе пять лет назад, сегодня может рассчитывать на стабильный доход от аренды и значительный рост стоимости объектов. Другой пример – парковочные места возле крупных торговых центров, таких как "Афимолл Сити" или "Мега Белая Дача". Здесь спрос на парковку всегда высокий, особенно в выходные дни и праздничные периоды, что обеспечивает высокий уровень доходности инвестиций.\n', '\n', 'Анна:\n', 'Отличные примеры. А как инвесторам оценить потенциальную доходность парковочного места перед покупкой?\n', '\n', 'Михаил:\n', 'Для оценки потенциальной доходности важно учесть несколько факторов. Во-первых, рассчитать все расходы, связанные с покупкой и обслуживанием парковочного места: цена покупки, налоги, затраты на ремонт и обслуживание, возможно, расходы на управление. Во-вторых, оценить потенциальный доход от аренды: изучить средние арендные ставки в выбранном районе и рассчитать ожидаемый ежемесячный и годовой доход. Затем можно использовать формулы расчета доходности, такие как доходность от аренды (годовой доход деленный на стоимость объекта). Также важно учесть возможный рост стоимости парковочного места со временем, что может увеличить общую доходность инвестиций.\n', '\n', 'Анна:\n', 'Это очень полезно. А какие ресурсы или инструменты вы рекомендуете использовать для анализа рынка парковочных мест?\n', '\n', 'Михаил:\n', 'Существует несколько полезных ресурсов и инструментов для анализа рынка парковочных мест. Во-первых, сайты недвижимости, такие как ЦИАН, Авито Недвижимость и Домклик, предоставляют актуальные данные о ценах и предложениях. Во-вторых, можно использовать аналитические отчеты и исследования от специализированных агентств, которые анализируют тенденции рынка. Также полезно посещать открытые мероприятия и семинары по недвижимости, где можно получить инсайдерскую информацию и советы от экспертов. Не забывайте о возможности консультаций с риэлторами и управляющими компаниями, которые имеют опыт работы с парковочными местами и могут предоставить ценные рекомендации.\n', '\n', 'Анна:\n', 'Спасибо, Михаил. На этом, пожалуй, мы подошли к концу нашего обсуждения. Есть ли у вас заключительные мысли по теме инвестиций в парковочные места в Москве?\n', '\n', 'Михаил:\n', 'Конечно. Инвестиции в парковочные места в Москве – это перспективное направление с высокой доходностью и относительно низкими рисками. Важно тщательно анализировать рынок, выбирать правильные локации и учитывать все возможные факторы, влияющие на спрос и стоимость. Также не стоит забывать о внедрении современных технологий и экологических решений, которые могут повысить привлекательность ваших парковочных мест. В целом, это отличный способ диверсифицировать свой инвестиционный портфель и обеспечить стабильный доход в будущем.\n', '\n', 'Анна:\n', 'Согласна с вами, Михаил. Спасибо за интересную и информативную беседу. Надеюсь, наши слушатели нашли её полезной и смогут применить полученные знания на практике. Благодарим всех за внимание и до новых встреч в нашем подкасте "Инвестиции в Москве"!\n', '\n', 'Михаил:\n', 'Спасибо, Анна! Всем удачных инвестиций и до свидания!']

### Response:
В Москве парковочные места являются привлекательным объектом для инвестиций. Это связано с постоянным спросом на них из-за нехватки парковочных мест в городе, а также с высокой доходностью от аренды. Инвестиции в парковочные места в Москве могут приносить стабильный доход с минимальными затратами на содержание. Однако, как и в любом инвестировании, здесь есть свои риски, такие как изменение законодательства, конкуренция и технические проблемы. Чтобы минимизировать риски, важно тщательно выбирать локацию, подписывать долгосроковые договоры аренды, следить за изменениями в законодательстве и регулярно обслуживать парковочные места. Доходность от инвестиций в парковочные места в Москве составляет от 8% до 12% годовых. Рынок парковочных мест в Москве будет продолжать расти в ближайшие 5-10 лет. Инвестиции в парковочные места в Москве можно сочетать с экологическими инициативами, например, путем покупки парковок с зарядными станциями для электромобилей. Цифровизация и автоматизация могут повышать эффективность управления парковочными местами и привлекать больше клиентов. При выборе парковочного места для инвестиций стоит учитывать его местоположение, инфраструктуру, юридическую чистоту и возможность расширения или модернизации. Также важно учитывать налоговое законодательство и налогообложение доходов от аренды. Для сравнения инвестиций в парковочные места можно рассмотреть другие виды недвижимости, такие как жилую или коммерческую. Жилая недвижимость требует больших вложений и имеет более длительные сроки окупаемости, но при этом предоставляет стабильный доход от аренды. Коммерческая недвижимость может предложить более высокую доходность, но сопряжена с большими рисками. Для начинающих инвесторов в парковочные места стоит начинать с покупки парковочных мест в хорошо зарекомендовавших себя районах с высоким спросом. Также стоит рассмотреть возможность сотрудничества с управляющими компаниями, которые помогут с арендой и техническим обслуживанием парковочных мест. Для управления парковочными местами можно использовать современные технологии, такие как IoT и умные системы управления, которые повышают эффективность и привлекательность парковок. Рост популярности электромобилей открывает новые возможности для инвесторов в парковочные места, оборудованные зарядными станциями. В долгосрочной перспективе переход на электромобили будет способствовать развитию инфраструктуры для их обслуживания. Существует несколько вариантов финансирования покупки парковочных мест: ипотечные кредиты, привлечение инвесторов или партнеров, использование собственных средств или реинвестирование прибыли от других инвестиций. Средняя цена парковочного места в Москве может варьироваться от 5 до 15 миллионов рублей в зависимости от района и типа парковки. Для оценки потенциальной доходности парковочного места нужно учесть все расходы, связанные с его покупкой и обслуживанием, а также рассчитать ожидаемый доход от аренды. Для анализа рынка парковочных мест можно использовать ресурсы и инструменты, такие как сайты недвижимости, аналитические отчеты и исследования, открытые мероприятия и семинары по недвижимости, консультации с риэлторами и управляющими компаниями.<|end_of_text|>


In [13]:
model.save_pretrained("model")
tokenizer.save_pretrained("model")

('model/tokenizer_config.json',
 'model/special_tokens_map.json',
 'model/tokenizer.json')